# Building a Naive Bayes Classifer

In [71]:
#import the necessary libraries
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import csv
import os
from collections import Counter

### The path of my input file. 
(to run on your system use the input path on your computer)

In [72]:
print(os.getcwd())

/Users/renitasequeira/Desktop/detect-ai-generated-essay


In [73]:
import os
for dirname, _, filenames in os.walk('/Users/renitasequeira/Desktop/Python/LLM - Detect AI generated text/input'):
    print("In here")
    for filename in filenames:
        print(os.path.join(dirname, filename))

In here
/Users/renitasequeira/Desktop/Python/LLM - Detect AI generated text/input/.DS_Store
/Users/renitasequeira/Desktop/Python/LLM - Detect AI generated text/input/train_prompts.csv
/Users/renitasequeira/Desktop/Python/LLM - Detect AI generated text/input/test_essays.csv
/Users/renitasequeira/Desktop/Python/LLM - Detect AI generated text/input/train_essays.csv
/Users/renitasequeira/Desktop/Python/LLM - Detect AI generated text/input/sample_submission.csv


## Reading the input files

In [222]:
df_train= pd.read_csv("./input/train_essays.csv")

In [75]:
test_data = pd.read_csv("./input/test_essays.csv")

In [76]:
train_data.head()

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0


In [77]:
test_data.head()

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [78]:
prompt_data = pd.read_csv("./input/train_prompts.csv")

In [12]:
prompt_data.head()

,prompt_id,prompt_name,instructions,source_text
0,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,1,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...


In [14]:
print(prompt_data)

   prompt_id                       prompt_name  \
0          0                   Car-free cities   
1          1  Does the electoral college work?   

                                        instructions  \
0  Write an explanatory essay to inform fellow ci...   
1  Write a letter to your state senator in which ...   

                                         source_text  
0  # In German Suburb, Life Goes On Without Cars ...  
1  # What Is the Electoral College? by the Office...  


In [ ]:
import csv
with open('./input/train_prompts.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        print(' '.join(row))

### Function to add rows to the train_essays.csv file
Text is generated on ChatGPT and the prompt_id specifies the prompt on which the essay is written.

In [20]:
def add_row_csv(text, prompt_id):
    import secrets

    random_id = '{:08x}'.format(secrets.randbelow(16**8))
    print(random_id)
    
#     text = """"""
    
    # Assuming the new data
    new_row = {
        'id': random_id,
        'prompt_id': prompt_id,
        'text': text,
        'generated': 1
    }

    # CSV file path
    csv_file_path = './input/train_essays.csv'

    # Open the file in append mode
    with open(csv_file_path, 'a', newline='') as csv_file:
        # Create a CSV writer object
        csv_writer = csv.DictWriter(csv_file, fieldnames=['id', 'prompt_id', 'text', 'generated'])

        # If the file is empty, write the header
        if csv_file.tell() == 0:
            csv_writer.writeheader()

        # Write the new row
        csv_writer.writerow(new_row)

    print("Row added successfully.")

In [113]:
add_row_csv(text, 1)

0638870d
Row added successfully.


In [115]:
add_row_csv(text, 1)

fe6a46b2
Row added successfully.


### Splitting the data into training and devlopment dataset

In [226]:
train_data = df_train.sample(frac = 0.9, random_state = 41)
dev_data = df_train.drop(train_data.index)
train_data.shape, dev_data.shape

((1254, 4), (139, 4))

In [27]:
train_data.head()

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0


## Function to build the vocabulary using the nltk kit

In [130]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Need to download them only once
# nltk.download('stopwords')
# nltk.download('punkt')

def build_vocabulary_with_nltk(essays):
    all_words = []

    for essay in essays:
        # Convert to lowercase for case-insensitivity
        essay_lower = essay.lower() if isinstance(essay, str) else str(essay)

        # Tokenize the essay into words
        words = word_tokenize(essay_lower)

        # Remove punctuation and numbers
        words = [word for word in words if word.isalpha()]

        # Remove stop words
        stop_words = set(stopwords.words('english'))
        words = [word for word in words if word not in stop_words]

        all_words.extend(words)

    # Remove duplicates to create a set of unique words
    unique_words = list(set(all_words))
    
    # Filter out rare words (occurrence less than five times)
    filtered_words = [word for word in unique_words if all_words.count(word) >= 5]

    # Sort the words for consistency
    filtered_words.sort()

    # Reverse index
    reverse_index = {word: index for index, word in enumerate(filtered_words)}

    return filtered_words, reverse_index


# Extract essays from the 'text' column
essays_from_dataset = train['text'].tolist()

# Build the vocabulary for the essay dataset using nltk
vocabulary_nltk_dataset, reverse_index_nltk_dataset = build_vocabulary_with_nltk(essays_from_dataset)

# Test
# print("Vocabulary List (with nltk) for the essay dataset:", vocabulary_nltk_dataset)
# print("Reverse Index (with nltk) for the essay dataset:", reverse_index_nltk_dataset)

In [131]:
len(vocabulary_nltk_dataset)

4097

### Function to count the documents containing the words from the vocabulary list

In [141]:
from collections import Counter

def count_documents_containing_words(word_list, documents):
    # Initialize a Counter to count the number of documents containing each word
    documents_containing_words = Counter()

    # Create a set for each word to efficiently check document presence
    word_presence = {word: set() for word in word_list}

    # Iterate over each document
    for doc_index, document in enumerate(documents):
        # Tokenize the document into words
        words = set(document.split())

        # Check and update the presence of each word in the current document
        for word in word_list:
            if word in words:
                word_presence[word].add(doc_index)

    # Count the number of documents containing each word
    for word, document_indices in word_presence.items():
        documents_containing_words[word] = len(document_indices)

    return documents_containing_words

#word list and the essays
word_list = vocabulary_nltk_dataset
documents = train['text'].tolist()

# Count the number of documents containing each word
documents_containing_words_count = count_documents_containing_words(word_list, documents)

# Display the result
# print("Number of Documents Containing Each Word:")
# print(documents_containing_words_count)

### Find the positive docs(LLM docs) count containing the words from the vocabulary list.

In [138]:
LLM_documents = train.loc[train.generated == 1]
#print(LLM_documents)


In [142]:
def count_LLM_documents_containing_words(word_list, documents):
    # Initialize a Counter to count the number of documents containing each word
    documents_containing_words = Counter()

    # Create a set for each word to efficiently check document presence
    word_presence = {word: set() for word in word_list}

    # Iterate over each document
    for doc_index, document in enumerate(documents):
        # Tokenize the document into words
        words = set(document.split())

        # Check and update the presence of each word in the current document
        for word in word_list:
            if word in words:
                word_presence[word].add(doc_index)

    # Count the number of documents containing each word
    for word, document_indices in word_presence.items():
        documents_containing_words[word] = len(document_indices)

    return documents_containing_words

#word list and the essays
word_list = vocabulary_nltk_dataset
documents = LLM_documents['text'].tolist()

# Count the number of documents containing each word
pos_documents_containing_words_count = count_documents_containing_words(word_list, documents)

# Display the result
# print("Number of Documents Containing Each Word:")
# print(documents_containing_words_count)

### Probability of the occurrence of the words in the vocabulary list
P[“the”] = num of documents containing ‘the’ / num of all documents


In [256]:
total_documents = len(train)

word_probabilities = {word: documents_containing_words_count[word] / total_documents for word in vocabulary_nltk_dataset}
# print("Probability of the occurrence for each word:")
# print(word_probabilities)

### Conditional probability based on the class (human or LLM)
P[“the” | LLM]  = # of positive documents containing “the” / num of all LLM documents


In [139]:
len(LLM_documents)


15

In [257]:
cond_probabilities = {word: pos_documents_containing_words_count[word] / len(LLM_documents) for word in vocabulary_nltk_dataset}
# print("Probability of the occurrence for each word:")
# print(word_probabilities)

## Model

In [212]:
len(vocab)

6157

In [255]:
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import nltk

# Load the dataset
df_train = pd.read_csv("./input/train_essays.csv")

# Split the dataset into train and development sets
train_data, dev_data = train_test_split(df_train, test_size=0.2, random_state=45)


# Step 3: Build a vocabulary and reverse index

# Download the NLTK stopwords if you haven't already
nltk.download('stopwords')

def build_vocab(data, min_occurrence=5):
    stop_words = set(stopwords.words('english'))

    word_counts = defaultdict(int)
    for essay in data['text']:
        for word in essay.split():
            # Exclude stopwords
            if word.lower() not in stop_words:
                word_counts[word] += 1
    
    vocab = [word for word, count in word_counts.items() if count >= min_occurrence]
    reverse_index = {word: idx for idx, word in enumerate(vocab)}
    
    return vocab, reverse_index

vocab, reverse_index = build_vocab(train_data)

# Calculate probabilities with Laplace smoothing
def calculate_probabilities(data, vocab, reverse_index, e=1):
    total_documents = len(data)
    class_counts = defaultdict(int)
    word_counts = defaultdict(int)
    class_word_counts = defaultdict(lambda: defaultdict(int))

    for _, row in data.iterrows():
        is_llm = row['generated']
        class_counts[is_llm] += 1
        for word in row['text'].split():
            word_id = reverse_index.get(word)
            if word_id is not None:
                word_counts[word_id] += 1
                class_word_counts[is_llm][word_id] += 1

    vocab_size = len(vocab)
    
    prior_prob = class_counts[True] / total_documents
    
    p_class_generated_true = class_counts[True]/total_documents
    probabilities = {}
    for word_id, word in enumerate(vocab):
        p_occurrence = (word_counts[word_id] + e) / (total_documents + e * vocab_size)
        p_occurrence_llm = (class_word_counts[True][word_id] + e) / (class_counts[True] + e * vocab_size)
        
        probabilities[word] = {'p_occurrence': p_occurrence, 'p_occurrence_llm': p_occurrence_llm}

    return probabilities, prior_prob

probabilities, prior_probability_llm_generated = calculate_probabilities(train_data, vocab, reverse_index)


# Calculate probabilities
# def calculate_probabilities(data, vocab, reverse_index):
#     total_documents = len(data)
#     class_counts = defaultdict(int)
#     word_counts = defaultdict(int)
#     class_word_counts = defaultdict(lambda: defaultdict(int))

#     for _, row in data.iterrows():
#         is_llm = row['generated']
#         class_counts[is_llm] += 1
#         for word in row['text'].split():
#             word_id = reverse_index.get(word)
#             if word_id is not None:
#                 word_counts[word_id] += 1
#                 class_word_counts[is_llm][word_id] += 1

#     probabilities = {}
#     for word_id, word in enumerate(vocab):
#         p_occurrence = word_counts[word_id] / total_documents
#         p_occurrence_llm = class_word_counts[True][word_id] / class_counts[True]
        
#         probabilities[word] = {'p_occurrence': p_occurrence, 'p_occurrence_llm': p_occurrence_llm}

#     return probabilities

# probabilities = calculate_probabilities(train_data, vocab, reverse_index)

# Calculate accuracy using dev dataset
def classify(essay, probabilities, prior_probability_llm_generated):
    score = 1.0
    score_denom1 = 1.0
    score_denom2 = 1.0
    for word in essay.split():
        word_id = reverse_index.get(word)
        if word_id is not None:
            score += probabilities[word]['p_occurrence_llm']
            score_denom1 *= probabilities[word]['p_occurrence_llm']
            score_denom2 *= (1-probabilities[word]['p_occurrence_llm'])
    return (score * prior_probability_llm_generated)/ (score_denom1 + score_denom2)
#     return score

correct_predictions = 0
for _, row in dev_data.iterrows():
    essay = row['text']
    is_llm = row['generated']
    prediction = classify(essay, probabilities, prior_probability_llm_generated)
    if (prediction >= 0.5 and is_llm) or (prediction < 0.5 and not is_llm):
        correct_predictions += 1

accuracy = correct_predictions / len(dev_data)
print(f"Accuracy on the development set: {accuracy}")


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/renitasequeira/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Accuracy on the development set: 0.989247311827957


In [241]:
# Sort the vocabulary based on conditional probability for LLM
sorted_vocab_llm = sorted(vocab, key=lambda word: probabilities[word]['p_occurrence_llm'], reverse=True)

# Sort the vocabulary based on conditional probability for human (not LLM)
sorted_vocab_human = sorted(vocab, key=lambda word: (1 - probabilities[word]['p_occurrence_llm']), reverse=True)

# Derive the top 10 words for each class
top_words_llm = sorted_vocab_llm[:10]
top_words_human = sorted_vocab_human[:10]

print("Top 10 words predicting LLM-generated essays:")
print(top_words_llm)

print("\nTop 10 words predicting human-generated essays:")
print(top_words_human)


Top 10 words predicting LLM-generated essays:
['Electoral', 'car', 'College', 'usage', 'urban', 'vote', 'candidates', 'electoral', 'limiting', 'cities']

Top 10 words predicting human-generated essays:
['congress', 'qualified', '538', '270', 'requires', 'elect', 'president.', 'held', '4', 'years']


In [243]:
# Load the test dataset
df_test = pd.read_csv("./input/test_essays.csv")

# Classify the test dataset
def classify_test(essay, probabilities, prior_probability_llm_generated):
    score = 1.0
    score_denom1 = 1.0
    score_denom2 = 1.0
    for word in essay.split():
        word_id = reverse_index.get(word)
        if word_id is not None:
            score *= probabilities[word]['p_occurrence_llm']
            score_denom1 *= probabilities[word]['p_occurrence_llm']
            score_denom2 *= (1-probabilities[word]['p_occurrence_llm'])
    return (score * prior_probability_llm_generated)/ (score_denom1 + score_denom2)

# Apply the classifier to the test dataset
df_test['predicted_generated'] = df_test['text'].apply(lambda x: classify_test(x, probabilities,prior_probability_llm_generated))

# Threshold for classification (adjust as needed)
# threshold = 0.5
# df_test['predicted_label'] = (df_test['predicted_generated'] >= threshold).astype(int)

# Save the results for Kaggle submission
submission_df = df_test[['id', 'predicted_generated']]
submission_df.to_csv("submission.csv", index=False)